In [1]:
from trout.intra.year import Year
from trout.intra.night import Night
from trout.nights import bad_nights
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def fwhm_for_year(year: int):
    y = Year(year)
    for n in y.nights:
        # Skip if the night is bad
        if n.is_bad():
            continue
        # If the night doesn't have color normalized folder
        # something could have gone wrong with the night
        if not n.has_color_normalized_folder():
            continue
        fwhm_for_night(n)

In [4]:
def get_good_image_numbers_for_night(night: Night):
    star_no, radius = 1, 4
    sample_flux_log = night.get_star_fluxlog_for_radius(star_no, radius)
    with open(sample_flux_log._path, "r") as fd:
        fd.readline()
        img_no = lambda : int(re.findall("\d+", fd.readline())[0])
        start, end = img_no(), img_no()
    return start, end

In [5]:
def fwhm_for_night(night: Night):
    # Use only the logfiles from the good part of the night
    start_img, end_img = get_good_image_numbers_for_night(night)
    # Noting that each logfile has XFWHM, YFWHM for each star, 
    # we'll find the median XFWHM vs X
    fwhm_x_lst = []
    x_lst = []
    for l in night.logfile_combined:
        if start_img <= l.image_number() <= end_img:
            fwhm_x_lst.append(l.data['XFWHM'])
            x_lst.append(l.data['X'])
    # Compute median
    median_x_fwhm = np.median(fwhm_x_lst, axis=0)
    median_x = np.median(x_lst, axis=0)
    plt.scatter(median_x, median_x_fwhm, s=1.5)
    plt.title(f"{night}")
    plt.xlabel("X")
    plt.ylabel("XFWHM")
    plt.show()

In [7]:
years = [2003, 2005, 2006, 2007, 2008]
# for year in years:
#     fwhm_for_year(year)